In [1]:
import numpy as np
import xlrd as pxl
import os
import glob
from image_util import *

In [ ]:
# generate datatset
input_edge = 256

for name in glob.glob("./data/PapSmearDatasetSingleCellNoSegmented/*/*"):
    pth = name.split('/')
    img = cv.imread(name)
    aug = augment_images(img, input_edge)
    pth[2] = 'PapSmearDatasetSingleCellAugmented'
    fl = pth[-1].split('.')
    name, ext = fl[0], fl[1]
    ext = '.' + ext
    for i in range(len(aug)):
        pth[-1] = name + '-aug' + str(i) + ext
        new_pth = '/'.join(pth)
        if not os.path.exists(os.path.dirname(new_pth)):
            os.makedirs(os.path.dirname(new_pth))
        cv.imwrite(new_pth, aug[i])

In [2]:
# read data - ONLY RUN ONCE
# load regressed nucleus/citoplasm ratios
def make_id(fname):
    return ''.join(fname.split('-')[:2])


valuesDict = dict()
sheet = pxl.open_workbook('./data/PapSmearDatasetSingleCell/PapSmearDatasetAnalysis.xls').sheet_by_index(0)
val_n = len(sheet.col(0))

for i in range(1, val_n):
    valuesDict[make_id(sheet.cell_value(i, 0))] = (sheet.cell_value(i, 3), sheet.cell_value(i, 21))


    
#load images
Xall = list() 
Yall_reg = list() 
Yall_cls = list() 

for pth in glob.glob("./data/PapSmearDatasetSingleCellAugmented/*/*"):
    img = cv.imread(pth)
    Id = make_id(os.path.basename(pth))
    Xall.append(img)
    r, c = valuesDict[Id]
    Yall_reg.append(r)
    Yall_cls.append(c)

Xall = np.array(Xall)
Yall_reg = np.array(Yall_reg)
Yall_cls = np.array(Yall_cls)

In [3]:
# load data
# Xtrain = np.load('./data/dataset256/Xtrain.npy')
Xtest = np.load('./data/dataset256/Xtest.npy')
# Ytrain_reg = np.load('./data/dataset256/Ytrain_reg.npy')
Ytest_reg = np.load('./data/dataset256/Ytest_reg.npy')
# Ytrain_cls = np.load('./data/dataset256/Ytrain_cls.npy')
Ytest_cls = np.load('./data/dataset256/Ytrain_cls.npy')

In [11]:
idx = list()
for i in range(len(Xtest)):
    for j in range(len(Xall)):
        if np.array_equal(Xtest[i], Xall[j]):
            idx.append(j)
            break
idx = np.array(idx)

In [15]:
Ytest_cls = Yall_cls[idx]
Ytest_reg  =Yall_reg[idx]

(3852, 256, 256, 3)

In [16]:
np.save('./data/dataset256/Ytest_reg', Ytest_reg)
np.save('./data/dataset256/Ytest_cls', Ytest_cls)

In [ ]:
#shuffle data - ONLY RUN ONCE
rand_perm = np.random.permutation(Xall.shape[0])
Xall = Xall[rand_perm]
Yall_reg = Yall_reg[rand_perm]
Yall_cls = Yall_cls[rand_perm]

In [ ]:
# split train and test - ONLY RUN ONCE
test_ratio = 0.2
cut = int(Xall.shape[0]*(1.-test_ratio))
Xtrain = Xall[:cut]
Xtest = Xall[cut:]
Ytrain_reg = Yall_reg[:cut]
Ytest_reg = Yall_reg[cut:]
Ytrain_cls = Yall_cls[:cut]
Ytest_cls = Yall_cls[cut:]

In [ ]:
#save data - ONLY RUN ONCE
np.save('./data/dataset256/Xtrain', Xtrain)
np.save('./data/dataset256/Xtest', Xtest)
np.save('./data/dataset256/Ytrain_reg', Ytrain_reg)
np.save('./data/dataset256/Ytest_reg', Ytest_reg)
np.save('./data/dataset256/Ytrain_cls', Ytrain_cls)
np.save('./data/dataset256/Ytest_cls', Ytest_cls)